In [63]:
import numpy as np
from sklearn import datasets
from abc import abstractmethod, ABCMeta

In [64]:
class CostFunction:
    
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def func(self, thetas):
        pass
    
    @abstractmethod
    def grad(self, thetas):
        pass
    
    @abstractmethod
    def line_search(self, thetas):
        pass
    
    @abstractmethod
    def dim(self):
        pass

    
class QuadraticCost(CostFunction):
    def __init__(self, A, B, C):
        self._A = A
        self._B = B
        self._C = C
    
    def func(self, thetas):
        return (0.5*thetas.T.dot(self._A).dot(thetas) + thetas.T.dot(self._B) + self._C).item()
    
    def grad(self, thetas):
        return self._A.dot(thetas) + self._B
    
    def line_search(self, thetas):
        grad = self.grad(thetas)
        return (grad.T.dot(grad)/(grad.T.dot(self._A).dot(grad))).item()
    
    def dim(self):
        return len(self._B)


class LinRegCost(QuadraticCost):
    def __init__(self, X, y):
        (m, n) = X.shape
        
        A = X.T.dot(X) / m
        B = - X.T.dot(y) / m
        C = - y.T.dot(y) / m
        
        QuadraticCost.__init__(self, A, B, C)


class GradientDescent:
    def __init__(self, cost, alpha=None):
        self.cost = cost
        self._const_alpha = alpha
    
    def alpha(self, theta):
        alpha_ = self._const_alpha
        if alpha_ is None:
            alpha_ = self.cost.line_search(theta)
        return alpha_
        
    def run(self, theta0, max_iter=1000, tol=1e-5):
        theta = theta0
        self.steps = [theta0]

        iter_ = 1 # variable to keep track of number of iteration
        while iter_ <= max_iter:
            # Gradient descent update
            theta_new = theta - self.alpha(theta) * self.cost.grad(theta)
            
            if abs(theta_new - theta).max() < tol:
                break
            else:
                theta = theta_new
            
            # Update number of iterations
            iter_ = iter_ + 1
            self.steps.append(theta)
        
        return theta
    
    def cost_at_steps(self):
        return [self.cost.func(theta) for theta in self.steps]


class LinearRegression:
    
    def normal_eq(self, X, y):
        self.thetas = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

    def gradient_descent(self, X, y):
        cost = LinRegCost(X, y)
        gd = GradientDescent(cost)
        self.thetas = gd.run(np.zeros((cost.dim(), 1)))
        
    def predict(self, X):
        return X.dot(self.thetas)

In [65]:
ds = datasets.load_boston()

In [66]:
mu = np.mean(ds.data, axis=0)
std = np.std(ds.data, axis=0)
X = (ds.data - mu) / std
(m, n) = X.shape
y = ds.target.reshape((m, 1))

In [67]:
lr = LinearRegression()
lr.normal_eq(X, y)
lr.thetas

array([[-0.92041113],
       [ 1.08098058],
       [ 0.14296712],
       [ 0.68220346],
       [-2.06009246],
       [ 2.67064141],
       [ 0.02112063],
       [-3.10444805],
       [ 2.65878654],
       [-2.07589814],
       [-2.06215593],
       [ 0.85664044],
       [-3.74867982]])

In [68]:
lr2 = LinearRegression()
lr2.gradient_descent(X, y)
lr2.thetas

array([[-0.92038364],
       [ 1.08093373],
       [ 0.14283377],
       [ 0.6822231 ],
       [-2.06006488],
       [ 2.67066438],
       [ 0.02110285],
       [-3.10446144],
       [ 2.6584445 ],
       [-2.07550172],
       [-2.06214093],
       [ 0.85663564],
       [-3.74866302]])